<a href="https://colab.research.google.com/github/yariasm54/Angular4_0/blob/master/Saber11_Ingles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as Pandas

#read data from Datos Abiertos
documentAddress = 'https://www.datos.gov.co/resource/tkn6-e4ic.json'
saber11DataFrame = Pandas.read_json(documentAddress)
saber11DataFrame.head(2)

,estu_tipodocumento,estu_nacionalidad,estu_genero,estu_fechanacimiento,periodo,estu_consecutivo,estu_estudiante,estu_pais_reside,estu_tieneetnia,estu_etnia,estu_depto_reside,estu_cod_reside_depto,estu_mcpio_reside,estu_cod_reside_mcpio,fami_estratovivienda,fami_personashogar,fami_cuartoshogar,fami_educacionpadre,fami_educacionmadre,fami_trabajolaborpadre,fami_trabajolabormadre,fami_tieneinternet,fami_tieneserviciotv,fami_tienecomputador,fami_tienelavadora,fami_tienehornomicroogas,fami_tieneautomovil,fami_tienemotocicleta,fami_tieneconsolavideojuegos,fami_numlibros,fami_comelechederivados,fami_comecarnepescadohuevo,fami_comecerealfrutoslegumbre,fami_situacioneconomica,estu_dedicacionlecturadiaria,estu_dedicacioninternet,estu_horassemanatrabaja,estu_tiporemuneracion,cole_codigo_icfes,cole_cod_dane_establecimiento,...,cole_naturaleza,cole_calendario,cole_bilingue,cole_caracter,cole_cod_dane_sede,cole_nombre_sede,cole_sede_principal,cole_area_ubicacion,cole_jornada,cole_cod_mcpio_ubicacion,cole_mcpio_ubicacion,cole_cod_depto_ubicacion,cole_depto_ubicacion,estu_privado_libertad,estu_cod_mcpio_presentacion,estu_mcpio_presentacion,estu_depto_presentacion,estu_cod_depto_presentacion,punt_lectura_critica,percentil_lectura_critica,desemp_lectura_critica,punt_matematicas,percentil_matematicas,desemp_matematicas,punt_c_naturales,percentil_c_naturales,desemp_c_naturales,punt_sociales_ciudadanas,percentil_sociales_ciudadanas,desemp_sociales_ciudadanas,punt_ingles,percentil_ingles,desemp_ingles,punt_global,percentil_global,estu_inse_individual,estu_nse_individual,estu_nse_establecimiento,estu_estadoinvestigacion,estu_generacion_e
0,TI,COLOMBIA,F,07/12/2000,20191,SB11201910008548,ESTUDIANTE,COLOMBIA,No,-,VALLE,76,CALI,76001,Estrato 4,3 a 4,Cuatro,Técnica o tecnológica incompleta,Secundaria (Bachillerato) completa,"Trabaja como profesional (por ejemplo médico, ...","Trabaja en el hogar, no trabaja o estudia",Si,Si,Si,Si,Si,No,No,No,0 A 10 LIBROS,Todos o casi todos los días,3 a 5 veces por semana,Todos o casi todos los días,Mejor,Entre 1 y 2 horas,Entre 1 y 3 horas,0,No,114686,176001002555,...,OFICIAL,A,N,TÉCNICO/ACADÉMICO,176001002555,INSTITUCIÓN EDUCATIVA TÉCNICA COMERCIAL HERNAN...,S,URBANO,NOCHE,76001,CALI,76,VALLE,N,76001,CALI,VALLE,76,50,29,2,49,31,2,47,33,2,40,22,1,54,39,A1,235,29,58.199483,3.0,3.0,PUBLICAR,NO
1,CC,COLOMBIA,F,23/03/1998,20191,SB11201910004475,ESTUDIANTE,COLOMBIA,No,-,VALLE,76,CALI,76001,Estrato 3,7 a 8,Cuatro,Técnica o tecnológica completa,Secundaria (Bachillerato) completa,Trabaja por cuenta propia (por ejemplo plomero...,"Trabaja en el hogar, no trabaja o estudia",Si,Si,No,Si,No,No,No,No,11 A 25 LIBROS,3 a 5 veces por semana,Todos o casi todos los días,Nunca o rara vez comemos eso,Mejor,No leo por entretenimiento,Entre 1 y 3 horas,0,No,114686,176001002555,...,OFICIAL,A,N,TÉCNICO/ACADÉMICO,176001002555,INSTITUCIÓN EDUCATIVA TÉCNICA COMERCIAL HERNAN...,S,URBANO,NOCHE,76001,CALI,76,VALLE,N,76001,CALI,VALLE,76,53,35,3,45,25,2,44,27,2,34,10,1,35,10,A-,217,21,51.018527,2.0,3.0,PUBLICAR,NO


Vamos a dividir las variables en varios tipos. Dependiendo si son variables binarias o personales ordinales (para efectos de una clasificación en árbol más adelantes)

In [0]:
#variables en archivo
variablesPersonalesBinarias = ['fami_tieneinternet', 'fami_tieneserviciotv',\
                     'fami_tienecomputador','fami_tienehornomicroogas',\
                     'fami_tieneautomovil', 'fami_tieneconsolavideojuegos']

variablesPersonalesOrdinales = ['fami_estratovivienda', \
                       'fami_educacionpadre','fami_educacionmadre', 'fami_trabajolaborpadre',\
                      'fami_trabajolabormadre','fami_numlibros', 'fami_comelechederivados',\
                      'fami_comecarnepescadohuevo', \
                      'fami_situacioneconomica',\
                      'estu_dedicacioninternet', 'estu_horassemanatrabaja',\
                      'estu_nse_individual']

#variables del examen
variablesExamen = ['percentil_lectura_critica', 'desemp_lectura_critica',\
                   'percentil_matematicas', 'desemp_matematicas',\
                   'percentil_c_naturales', 'desemp_c_naturales',\
                   'percentil_sociales_ciudadanas',\
                   'desemp_sociales_ciudadanas',  'percentil_ingles',\
                   'percentil_global','estu_inse_individual']
#variables de puntajes
puntajesExamen = ['punt_matematicas','punt_c_naturales','punt_sociales_ciudadanas','punt_ingles','punt_lectura_critica']

#variables del colegio
variablesColegio = ['cole_naturaleza','cole_bilingue','cole_caracter','cole_genero','cole_calendario','cole_jornada']
#variables objetivo aqui
variableObjetivo = ['desemp_ingles','punt_global']

Analizar variables para ver consistencia e inconsistencia


In [0]:
for varBinaria in variablesPersonalesBinarias:
  resultados = saber11DataFrame[varBinaria].value_counts()
  print(varBinaria)
  print(resultados)

fami_tieneinternet
Si    770
No    152
-      78
Name: fami_tieneinternet, dtype: int64
fami_tieneserviciotv
Si    795
No    128
-      77
Name: fami_tieneserviciotv, dtype: int64
fami_tienecomputador
Si    739
No    209
-      52
Name: fami_tienecomputador, dtype: int64
fami_tienehornomicroogas
Si    665
No    280
-      55
Name: fami_tienehornomicroogas, dtype: int64
fami_tieneautomovil
No    479
Si    465
-      56
Name: fami_tieneautomovil, dtype: int64
fami_tieneconsolavideojuegos
No    515
Si    426
-      59
Name: fami_tieneconsolavideojuegos, dtype: int64


Analisamos las variables ordinales

In [0]:
for varOrdinal in variablesPersonalesOrdinales:
  resultados = saber11DataFrame[varOrdinal].value_counts()
  print(varOrdinal)
  print(resultados)

fami_estratovivienda
Estrato 3      240
Estrato 2      209
Estrato 6      135
Estrato 4      123
Estrato 1      100
Estrato 5       92
-               81
Sin Estrato     20
Name: fami_estratovivienda, dtype: int64
fami_educacionpadre
Educación profesional completa          205
Postgrado                               155
Secundaria (Bachillerato) completa      152
Primaria incompleta                      95
-                                        76
Técnica o tecnológica completa           66
Secundaria (Bachillerato) incompleta     62
No sabe                                  55
Primaria completa                        52
Ninguno                                  40
Educación profesional incompleta         16
Técnica o tecnológica incompleta         14
No Aplica                                12
Name: fami_educacionpadre, dtype: int64
fami_educacionmadre
Educación profesional completa          239
Secundaria (Bachillerato) completa      174
Postgrado                               133
Pr

Vamos a retirar de la base de datos todos los '-'

In [0]:
variablesALimpiar = variablesPersonalesBinarias + variablesPersonalesOrdinales
registrosConProblemas = []
for indice in range(len(saber11DataFrame)):
  contieneMenos = False
  for var in variablesALimpiar:
    contieneMenos = contieneMenos or saber11DataFrame.iloc[indice][var] == '-'
  if contieneMenos:
    registrosConProblemas.append(indice)
print('Registros con problemas: ', len(registrosConProblemas))

Registros con problemas:  148


In [0]:
saber11DataFrame = saber11DataFrame.drop(saber11DataFrame.index[registrosConProblemas])


In [0]:
saber11DataFrame

,estu_tipodocumento,estu_nacionalidad,estu_genero,estu_fechanacimiento,periodo,estu_consecutivo,estu_estudiante,estu_pais_reside,estu_tieneetnia,estu_etnia,estu_depto_reside,estu_cod_reside_depto,estu_mcpio_reside,estu_cod_reside_mcpio,fami_estratovivienda,fami_personashogar,fami_cuartoshogar,fami_educacionpadre,fami_educacionmadre,fami_trabajolaborpadre,fami_trabajolabormadre,fami_tieneinternet,fami_tieneserviciotv,fami_tienecomputador,fami_tienelavadora,fami_tienehornomicroogas,fami_tieneautomovil,fami_tienemotocicleta,fami_tieneconsolavideojuegos,fami_numlibros,fami_comelechederivados,fami_comecarnepescadohuevo,fami_comecerealfrutoslegumbre,fami_situacioneconomica,estu_dedicacionlecturadiaria,estu_dedicacioninternet,estu_horassemanatrabaja,estu_tiporemuneracion,cole_codigo_icfes,cole_cod_dane_establecimiento,...,cole_naturaleza,cole_calendario,cole_bilingue,cole_caracter,cole_cod_dane_sede,cole_nombre_sede,cole_sede_principal,cole_area_ubicacion,cole_jornada,cole_cod_mcpio_ubicacion,cole_mcpio_ubicacion,cole_cod_depto_ubicacion,cole_depto_ubicacion,estu_privado_libertad,estu_cod_mcpio_presentacion,estu_mcpio_presentacion,estu_depto_presentacion,estu_cod_depto_presentacion,punt_lectura_critica,percentil_lectura_critica,desemp_lectura_critica,punt_matematicas,percentil_matematicas,desemp_matematicas,punt_c_naturales,percentil_c_naturales,desemp_c_naturales,punt_sociales_ciudadanas,percentil_sociales_ciudadanas,desemp_sociales_ciudadanas,punt_ingles,percentil_ingles,desemp_ingles,punt_global,percentil_global,estu_inse_individual,estu_nse_individual,estu_nse_establecimiento,estu_estadoinvestigacion,estu_generacion_e
0,TI,COLOMBIA,F,07/12/2000,20191,SB11201910008548,ESTUDIANTE,COLOMBIA,No,-,VALLE,76,CALI,76001,Estrato 4,3 a 4,Cuatro,Técnica o tecnológica incompleta,Secundaria (Bachillerato) completa,"Trabaja como profesional (por ejemplo médico, ...","Trabaja en el hogar, no trabaja o estudia",Si,Si,Si,Si,Si,No,No,No,0 A 10 LIBROS,Todos o casi todos los días,3 a 5 veces por semana,Todos o casi todos los días,Mejor,Entre 1 y 2 horas,Entre 1 y 3 horas,0,No,114686,176001002555,...,OFICIAL,A,N,TÉCNICO/ACADÉMICO,176001002555,INSTITUCIÓN EDUCATIVA TÉCNICA COMERCIAL HERNAN...,S,URBANO,NOCHE,76001,CALI,76,VALLE,N,76001,CALI,VALLE,76,50,29,2,49,31,2,47,33,2,40,22,1,54,39,A1,235,29,58.199483,3.0,3.0,PUBLICAR,NO
1,CC,COLOMBIA,F,23/03/1998,20191,SB11201910004475,ESTUDIANTE,COLOMBIA,No,-,VALLE,76,CALI,76001,Estrato 3,7 a 8,Cuatro,Técnica o tecnológica completa,Secundaria (Bachillerato) completa,Trabaja por cuenta propia (por ejemplo plomero...,"Trabaja en el hogar, no trabaja o estudia",Si,Si,No,Si,No,No,No,No,11 A 25 LIBROS,3 a 5 veces por semana,Todos o casi todos los días,Nunca o rara vez comemos eso,Mejor,No leo por entretenimiento,Entre 1 y 3 horas,0,No,114686,176001002555,...,OFICIAL,A,N,TÉCNICO/ACADÉMICO,176001002555,INSTITUCIÓN EDUCATIVA TÉCNICA COMERCIAL HERNAN...,S,URBANO,NOCHE,76001,CALI,76,VALLE,N,76001,CALI,VALLE,76,53,35,3,45,25,2,44,27,2,34,10,1,35,10,A-,217,21,51.018527,2.0,3.0,PUBLICAR,NO
2,TI,COLOMBIA,M,22/03/2001,20191,SB11201910011427,ESTUDIANTE,COLOMBIA,No,-,VALLE,76,CALI,76001,Estrato 2,5 a 6,Cuatro,Primaria incompleta,Educación profesional completa,Es dueño de un negocio pequeño (tiene pocos em...,Tiene un trabajo de tipo auxiliar administrati...,Si,Si,No,Si,Si,No,No,Si,0 A 10 LIBROS,3 a 5 veces por semana,Nunca o rara vez comemos eso,Nunca o rara vez comemos eso,Igual,No leo por entretenimiento,30 minutos o menos,Más de 30 horas,"Si, en efectivo",114686,176001002555,...,OFICIAL,A,N,TÉCNICO/ACADÉMICO,176001002555,INSTITUCIÓN EDUCATIVA TÉCNICA COMERCIAL HERNAN...,S,URBANO,NOCHE,76001,CALI,76,VALLE,N,76001,CALI,VALLE,76,60,54,3,52,38,3,44,27,2,41,23,2,49,32,A1,246,34,50.855657,2.0,3.0,PUBLICAR,NO
3,CC,COLOMBIA,M,16/06/1994,20191,SB11201910041975,ESTUDIANTE,COLOMBIA,No,-,VALLE,76,CALI,76001,Estrato 2,3 a 4,Dos,Educación profesional completa,Educación profesional completa,"Trabaja como profesional (por ejemplo médico, ..."

Subdividir variables

In [0]:
caractaristicas = ['fami_tieneinternet','fami_numlibros','estu_nse_individual']
variableObjetivo = 'desemp_ingles'
subSaber11DataFrame = saber11DataFrame[caractaristicas + [variableObjetivo]]
subSaber11DataFrame

,fami_tieneinternet,fami_numlibros,estu_nse_individual,desemp_ingles
0,Si,0 A 10 LIBROS,3.0,A1
1,Si,11 A 25 LIBROS,2.0,A-
2,Si,0 A 10 LIBROS,2.0,A1
3,Si,0 A 10 LIBROS,3.0,A2
4,Si,MÁS DE 100 LIBROS,4.0,B1
...,...,...,...,...
995,No,0 A 10 LIBROS,1.0,A1
996,Si,0 A 10 LIBROS,3.0,A2
997,Si,0 A 10 LIBROS,4.0,A2
998,Si,0 A 10 LIBROS,4.0,A-


Vamos a reemplazar Si y No por numeros 1 y 0

In [0]:
subSaber11DataFrame['fami_tieneinternet'] = subSaber11DataFrame['fami_tieneinternet'].map({'Si':1,'No':0})
subSaber11DataFrame.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,fami_tieneinternet,fami_numlibros,estu_nse_individual,desemp_ingles
0,1,0 A 10 LIBROS,3.0,A1
1,1,11 A 25 LIBROS,2.0,A-
2,1,0 A 10 LIBROS,2.0,A1
3,1,0 A 10 LIBROS,3.0,A2
4,1,MÁS DE 100 LIBROS,4.0,B1


In [0]:
def generarDiccionarioCategorias(nomVariable):
  valores = subSaber11DataFrame[nomVariable].value_counts()
  print(valores)
  for valor in valores.iteritems():
    print(valor[0], 'tiene', valor[1])
diccionarioCantidadDeLibros = {
    '0 A 10 LIBROS' : 1,
    '11 A 25 LIBROS' : 2,
    '26 A 100 LIBROS' : 3,
    'MÁS DE 100 LIBROS' : 4
}
diccionarioNivelesDeIngles = {
    'A-' : 1,
    'A1' : 2,
    'A2' : 3,
    'B1' : 4,
    'B+' : 5,
}
generarDiccionarioCategorias('desemp_ingles')

A-    302
B+    201
A1    150
B1    117
A2     82
Name: desemp_ingles, dtype: int64
A- tiene 302
B+ tiene 201
A1 tiene 150
B1 tiene 117
A2 tiene 82


In [0]:
subSaber11DataFrame['fami_numlibros'] = subSaber11DataFrame['fami_numlibros'].map(diccionarioCantidadDeLibros)
subSaber11DataFrame.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,fami_tieneinternet,fami_numlibros,estu_nse_individual,desemp_ingles
0,1,1,3.0,A1
1,1,2,2.0,A-
2,1,1,2.0,A1
3,1,1,3.0,A2
4,1,4,4.0,B1


In [0]:
subSaber11DataFrame['desemp_ingles'] = subSaber11DataFrame['desemp_ingles'].map(diccionarioNivelesDeIngles)
subSaber11DataFrame.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,fami_tieneinternet,fami_numlibros,estu_nse_individual,desemp_ingles
0,1,1,3.0,2
1,1,2,2.0,1
2,1,1,2.0,2
3,1,1,3.0,3
4,1,4,4.0,4


Agregar codigo desde https://www.datacamp.com/community/tutorials/decision-tree-classification-python

In [0]:
# Load libraries
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

## Modificar el siguiente codigo:

X = subSaber11DataFrame[caractaristicas] # Features
y = subSaber11DataFrame[variableObjetivo] # Target variable

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test



In [0]:

# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

print(clf)
print(y_pred)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
[1 1 5 1 1 1 1 5 1 1 5 1 1 5 1 1 1 1 5 5 5 1 1 4 1 1 5 1 2 1 1 5 1 1 1 1 5
 1 5 1 1 1 1 5 1 1 1 5 1 5 5 1 1 1 5 1 5 1 2 5 4 1 1 1 5 1 1 5 1 5 1 4 5 5
 5 5 5 5 1 1 5 1 1 5 1 1 1 1 1 1 1 1 5 1 1 1 1 1 5 5 5 1 1 1 5 1 1 1 1 4 1
 1 1 5 1 5 5 1 5 5 1 2 5 4 1 1 1 4 5 1 1 1 1 1 5 5 5 5 5 5 2 1 1 1 1 5 3 1
 1 1 5 5 5 5 5 1 3 1 1 1 5 1 5 5 4 1 1 1 1 1 5 1 1 5 5 5 1 1 5 1 1 3 1 1 1
 5 1 4 1 5 1 1 1 1 5 1 5 1 1 5 1 1 1 5 1 2 1 5 5 5 5 1 1 4 1 1 5 5 1 1 5 1
 5 1 5 1 1 5 5 5 1 1 3 5 1 5 5 5 1 5 5 5 1 5 1 4 5 5 1 1 1 1 5 1 1 5]
